# Spell Checking using Norvig's Algorithm

## Importing the Python regex and Counter libraries to extract words containing only alphabets from 'big.txt'

In [1]:
import re
from collections import Counter

# using regular expression r'\w+'
def extraction(text): 
    return re.findall(r'\w+', text.lower())

Dict_of_words = Counter(extraction(open('big.txt').read()))

## Takes a word as an input and returns the probabiblity of that word

In [2]:
def P(word): 
    # Number of times the input word has come in the big.txt / Total words in the big.txt
    N=sum(Dict_of_words.values())
    return Dict_of_words[word] / N

In [3]:
P('how')

0.001178753748033543

## Takes a word as an input and returns all the words that are 1 edit distance away

In [4]:
def edits1(word):
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

In [5]:
# edits1('how')

## Takes a word as an input and returns all the words that are 2 edit distance away

In [6]:
def edits2(word): 
    return set(e2 for e1 in edits1(word) for e2 in edits1(e1))

In [7]:
# edits2('how')

## Takes a list of words and returns a subset of the list of words that are present in the WORDS dictionary

In [8]:
def known(words): 
    return set(w for w in words if w in Dict_of_words)

## Takes a word as an input and returns all the possible correct candidates for that word 

In [9]:
def candidates(word): 
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

## Takes a word as an input and returns the correct word with the highest probability

In [10]:
def correction(word): 
    return max(candidates(word), key=P)

## Testing the correction function

In [11]:
print(correction('whet'))
print(correction('hapy'))
print(correction('diad'))
print(correction('Manipolate'))

what
happy
did
manipulate


# Spell Checking using the SymSpell delete spell checking algorithm

## Importing the Python regex and Counter libraries to extract words containing only alphabets from 'big.txt'

In [12]:
import re
from collections import Counter

def words(text): return re.findall('[a-z]+', text.lower())

WORDS = dict(Counter(words(open('big.txt').read())))
Edits1 = {}
Edits2 = {}

## Returns a set of words formed by deleting 1 character from the word

In [13]:
def delete(word):
    splits     = [(word[:i], word[i:]) for i in range(len(word) + 1)]
    deletes    = [L + R[1:] for L, R in splits if R]
    return set(deletes)

## Fills the values in the two dictionaries edits1 and edits2, edits1 contain words that can be formed by deleting 1 character from the input word and edits2 contain words that can be formed by deleting two characters from the input word.

In [14]:
def populate(word):
    set1 = delete(word)
    set2 = set([e2 for e1 in set1 for e2 in delete(e1)])
    for x in set1:
        Edits1.setdefault(x,[]).append(word)
    for x in set2:
        Edits2.setdefault(x,[]).append(word)

## Populating the Edits1 and Edits2 dictionaries

In [15]:
for w in WORDS.keys():
    populate(w)

## The function will take the input word and return the correct word, it will generate all the words that can be formed by deleting one or two characters from the input word. Then it finds the words corresponding to the words found in Edits1 and Edits2 dictionaries and finds the most common words in the WORDS dictionary

In [16]:
def correction(inp):
    a = None
    b = None
    if inp in WORDS.keys():
        return inp
    s1 = list(delete(inp))+[inp]
    s2 = []
    for x in s1:
        if x in Edits1.keys():
            s2 += Edits1[x]
    if len(s2):
        a = max(s2, key=WORDS.get)
    
    s1 = set(s1)
    s2 = set([e2 for e1 in s1 for e2 in delete(e1)])
    s3 = list(s1|s2)+[inp]
    s4 = []
    for x in s3:
        if x in Edits2.keys():
            s4 += Edits2[x]
    if len(s4):
        b = max(s4, key=WORDS.get)
    if a and a!=b:
        return (a,b)
    if b:
        return b
    else:
        return 'No candidate found'

## Testing the correction function, returns 1 or 2 correct words or 'No candidate found' if nothing is found

In [17]:
print(correction('whet'))
print(correction('hapy'))
print(correction('diad'))
print(correction('Manipolate'))

('what', 'that')
('happy', 'that')
('dead', 'said')
manipulate


# SymSpell Algorithm is million times faster than Norvig's Algorithm